In [146]:
import pandas as pd
import yfinance as yf

In [147]:
df = pd.read_parquet('cleaned_options_SPX.parquet')  
print(df.head())
print(df.info())
print(df.isnull().sum())
print(df.describe())
print(df.tail())

  ticker        date      exdate cp_flag  open_interest  contract_size  \
0    SPX  2018-08-31  2018-09-21       C             65            100   
1    SPX  2018-08-31  2018-09-21       C             22            100   
2    SPX  2018-08-31  2018-09-21       C             82            100   
3    SPX  2018-08-31  2018-09-21       C           1096            100   
4    SPX  2018-08-31  2018-09-21       C             11            100   

      gamma  
0  0.000020  
1  0.000030  
2  0.000016  
3  0.000039  
4  0.000048  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13569891 entries, 0 to 13569890
Data columns (total 7 columns):
 #   Column         Dtype  
---  ------         -----  
 0   ticker         object 
 1   date           object 
 2   exdate         object 
 3   cp_flag        object 
 4   open_interest  int64  
 5   contract_size  int64  
 6   gamma          float64
dtypes: float64(1), int64(2), object(4)
memory usage: 724.7+ MB
None
ticker           0
date             

In [148]:
spy = yf.download('SPY', start=df['date'].min(), end=df['date'].max() + pd.Timedelta(days=1), auto_adjust=True)
spy.columns = ['_'.join(col).strip() for col in spy.columns.values]
spy.reset_index(inplace = True)
spy = spy.rename(columns = {'Date':'date', 'Close_SPY': 'Pt'})
spy = spy[['date','Pt', 'Volume_SPY']]
df['date'] = pd.to_datetime(df['date'])
spy['date'] = pd.to_datetime(spy['date'])
df = pd.merge(df, spy, on='date', how='left')
spy

[*********************100%***********************]  1 of 1 completed


,date,Pt,Volume_SPY
0,2018-08-31,259.882141,66140800
1,2018-09-04,259.434570,57594400
2,2018-09-05,258.736298,72452400
3,2018-09-06,257.957489,65909900
4,2018-09-07,257.456207,73524800
...,...,...,...
1253,2023-08-25,428.563019,102325100
1254,2023-08-28,431.280731,61595400
1255,2023-08-29,437.514771,83081900
1256,2023-08-30,439.316833,69053900


In [152]:
# Calculate Gamma Dollar
df['gamma_dollar'] = df.apply(
    lambda row: row['gamma'] * row['open_interest'] * row['contract_size'] * row['Pt']
    if row['cp_flag'] == 'C'
    else -row['gamma'] * row['open_interest'] * row['contract_size'] * row['Pt'],
    axis=1
)

In [154]:
df

,ticker,date,exdate,cp_flag,open_interest,contract_size,gamma,Pt,Volume_SPY,gamma_dollar
0,SPX,2018-08-31,2018-09-21,C,65,100,0.000020,259.882141,66140800,33.784678
1,SPX,2018-08-31,2018-09-21,C,22,100,0.000030,259.882141,66140800,17.152221
2,SPX,2018-08-31,2018-09-21,C,82,100,0.000016,259.882141,66140800,34.096537
3,SPX,2018-08-31,2018-09-21,C,1096,100,0.000039,259.882141,66140800,1110.840224
4,SPX,2018-08-31,2018-09-21,C,11,100,0.000048,259.882141,66140800,13.721777
...,...,...,...,...,...,...,...,...,...,...
13569886,SPX,2023-08-31,2024-06-28,P,39,100,0.000664,438.673920,66084600,-1135.989982
13569887,SPX,2023-08-31,2024-06-28,P,2,100,0.000608,438.673920,66084600,-53.342749
13569888,SPX,2023-08-31,2024-06-28,P,2,100,0.000529,438.673920,66084600,-46.411701
13569889,SPX,2023-08-31,2024-06-28,P,18,100,0.000440,438.673920,66084600,-347.429744


In [164]:
daily_nge = df.groupby('date')['gamma_dollar'].sum().reset_index(name='NGE_raw')
SPY_SHARES_OUTSTANDING = 950000000
daily_nge = pd.merge(daily_nge, df[['date', 'Pt']].drop_duplicates(), on='date', how='left')
daily_nge['MV_t'] = daily_nge['Pt'] * SPY_SHARES_OUTSTANDING
daily_nge['NGE'] = daily_nge['NGE_raw'] / daily_nge['MV_t']
daily_nge

,date,NGE_raw,Pt,MV_t,NGE
0,2018-08-31,1.573815e+08,259.882141,2.468880e+11,0.000637
1,2018-09-04,1.395750e+08,259.434570,2.464628e+11,0.000566
2,2018-09-05,1.080051e+08,258.736298,2.457995e+11,0.000439
3,2018-09-06,7.010234e+07,257.957489,2.450596e+11,0.000286
4,2018-09-07,2.563834e+07,257.456207,2.445834e+11,0.000105
...,...,...,...,...,...
1253,2023-08-25,-5.343755e+07,428.563019,4.071349e+11,-0.000131
1254,2023-08-28,-2.095173e+07,431.280731,4.097167e+11,-0.000051
1255,2023-08-29,3.688119e+07,437.514771,4.156390e+11,0.000089
1256,2023-08-30,5.135533e+07,439.316833,4.173510e+11,0.000123


In [174]:
daily_nge.to_csv('daily_nge.csv', index=False)